In [1]:
!ollama pull llama3.1
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏ 1.6 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏ 5.1 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏ 6.8 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏  11 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏  15 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏  17 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏  21 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc:   0% ▕                  ▏  22 MB/4

In [2]:
!pip install langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_ollama-0.3.10-py3-none-any.whl.metadata (2.1 kB)


In [3]:
from langchain_ollama import ChatOllama

# Initialize the model
llm = ChatOllama(model="llama3.1")

# Test it
response = llm.invoke("Hello! Are you running locally?")
print(response.content)

/Users/rubesh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


I'm a cloud-based AI, so I don't have any local processes running on your device. When you interact with me, your input is sent to our servers where it's processed and responded to in real-time. How can I assist you today?


In [4]:
import os
import csv
from langchain_community.vectorstores import Chroma
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# --- NEW IMPORTS FOR OLLAMA ---
from langchain_ollama import ChatOllama, OllamaEmbeddings

# CONFIGURATION 

In [12]:

DB_PATH = "chroma_db"
LEADS_FILE = "student_leads.csv"

# Create leads file if it doesn't exist
if not os.path.exists(LEADS_FILE):
    with open(LEADS_FILE, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Email", "Interest", "Timestamp"])

# DEFINE TOOLS 

In [7]:

@tool
def retrieve_school_info(query: str):
    """
    Use this tool to find information about ESILV programs, courses, admission, 
    exams, calendars, and student life from the internal documentation.
    """
    print(f" [Ollama] Searching database for: '{query}'...") 
    try:
        # SWITCHED TO OLLAMA EMBEDDINGS
        embeddings = OllamaEmbeddings(model="nomic-embed-text")
        
        # Connect to DB
        vectorstore = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        
        docs = retriever.invoke(query)
        if not docs:
            return "No documents found in the database."
            
        return "\n\n".join([d.page_content for d in docs])
    except Exception as e:
        return f"Error during retrieval: {str(e)}"

@tool
def save_contact_info(name: str, email: str, interest: str = "General"):
    """
    Use this tool ONLY when a user explicitly provides their name and email 
    for registration, follow-up, or more information.
    """
    import datetime
    print(f"💾 [Ollama] Saving lead: {name} ({email})...") 
    try:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(LEADS_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([name, email, interest, timestamp])
        return f"Success! Contact details for {name} have been saved."
    except Exception as e:
        return f"Error saving file: {str(e)}"

# INITIALIZE AGENT

In [8]:


print("⏳ Initializing Ollama (Llama 3.1)... This might take a moment.")

# SWITCHED TO CHAT OLLAMA
# We use "llama3.1" because it has excellent support for Tool Calling.
llm = ChatOllama(
    model="llama3.1",
    temperature=0
)

tools = [retrieve_school_info, save_contact_info]

⏳ Initializing Ollama (Llama 3.1)... This might take a moment.


# PROMPT (Standard Agent Prompt)

In [9]:

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are the official AI Assistant for ESILV Engineering School. "
     "Your goal is to help students with information and collect leads. "
     "1. If the user asks a question, use the 'retrieve_school_info' tool. "
     "2. If the user wants to apply or be contacted, ask for their Name and Email. "
     "3. Once you have Name and Email, use the 'save_contact_info' tool. "
     "4. Always be polite and professional."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"), 
])

agent = create_tool_calling_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ Ollama Agent is ready!")

✅ Ollama Agent is ready!


In [10]:
print("\n--- TEST 1: Retrieval ---")
try:
    # Note: Ollama might be slower than Gemini, so be patient
    response = agent_executor.invoke({"input": "What is the difference between Year 3, Year 4, and Year 5 in the engineering cycle?"})
    print(f"Answer: {response['output']}")
except Exception as e:
    print(f"Test 1 Failed: {e}")


--- TEST 1: Retrieval ---


> Entering new AgentExecutor chain...

Invoking: `retrieve_school_info` with `{'query': 'difference between Year 3, Year 4, and Year 5 in the engineering cycle'}`


🔍 [Ollama] Searching database for: 'difference between Year 3, Year 4, and Year 5 in the engineering cycle'...


/var/folders/xj/0g1kw6bx68b3xd_90h42bzh40000gn/T/ipykernel_47021/3662941420.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)


(La mise en œuvre des prestations d’assistance est confiée à Mondial Assistance France) 
Vos références complètes à communiquer lors votre appel ........................ 60 513 743 / 921773 
Téléphone .......................................................................................................(33 1) 01.40.25.58.80 
Télécopie .........................................................................................................(33 1) 01.42.99.03.00 
 
Avant d’appeler, munissez-vous des renseignements suivants : 
 Le numéro de contrat (n°60 513 743), 
 Le numéro de téléphone où l’on peut vous joindre, 
 L’adresse exacte où vous vous trouvez, 
 Le nom, l’adresse et le numéro du médecin traitant et/ou ceux de l’hôpital, 
 Qui a besoin d’aide, 
 Où et pourquoi. 
 
 
 
 
 
 
 
 
 
Allianz IARD – Entreprise régie par le Code des assurances.

St Francis CollegeEtats-Unishttps://www.sfc.edu/


California State University, Los AngelesĂ‰tats-Unishttp://www.calstatela.edu/


Manh

In [11]:
print("\n--- TEST 2: Lead Capture ---")
try:
    response = agent_executor.invoke({"input": "I want to register. My name is Rubesh and email is rubesh@test.com"})
    print(f"Answer: {response['output']}")
except Exception as e:
    print(f"Test 2 Failed: {e}")


--- TEST 2: Lead Capture ---


> Entering new AgentExecutor chain...

Invoking: `save_contact_info` with `{'email': 'rubesh@test.com', 'interest': 'Registration', 'name': 'Rubesh', 'query': ''}`


💾 [Ollama] Saving lead: Rubesh (rubesh@test.com)...
Success! Contact details for Rubesh have been saved.I'll make sure to keep you updated on any new information or opportunities related to ESILV Engineering School. If you have any other questions or need further assistance, feel free to ask!

> Finished chain.
Answer: I'll make sure to keep you updated on any new information or opportunities related to ESILV Engineering School. If you have any other questions or need further assistance, feel free to ask!
